In [1]:
import torch
import numpy as np
from tqdm import tqdm

In [4]:

# Define alphas as torch tensors with requires_grad=True
alpha_1 = torch.tensor(1.013, dtype=torch.float64, requires_grad=True)
alpha_2 = torch.tensor(0.2119, dtype=torch.float64, requires_grad=True)
alpha_3 = torch.tensor(0.1406, dtype=torch.float64, requires_grad=True)
alpha_4 = torch.tensor(0.003, dtype=torch.float64, requires_grad=True)

alphas = [alpha_1, alpha_2, alpha_3, alpha_4]

# psi function using torch
def psi(x, y, alpha_1, alpha_2, alpha_3, alpha_4):
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

# Tensors with gradients enabled
x = torch.tensor([0.2, 0.3, 0.1], dtype=torch.float64, requires_grad=True)
y = torch.tensor([0.1, 0.4, 0.3], dtype=torch.float64, requires_grad=True)

# Compute psi
psi_val = psi(x, y, *alphas)

# First gradients w.r.t. x and y
grad_x = torch.autograd.grad(psi_val, x, create_graph=True)[0]
grad_y = torch.autograd.grad(psi_val, y, create_graph=True)[0]

# Laplacians
laplacian_x = sum(torch.autograd.grad(grad_x[i], x, retain_graph=True, create_graph=True)[0][i] for i in range(3))
laplacian_y = sum(torch.autograd.grad(grad_y[i], y, retain_graph=True, create_graph=True)[0][i] for i in range(3))

lap = laplacian_x + laplacian_y
# Derivatives of Laplacians w.r.t. alphas
dlap_dalpha = torch.autograd.grad(lap, alphas, retain_graph=True)


print(dlap_dalpha)

(tensor(-0.5851, dtype=torch.float64), tensor(2.5131, dtype=torch.float64), tensor(0.6771, dtype=torch.float64), tensor(-2.5998, dtype=torch.float64))


In [5]:
"""
def local_energy(psi_val, r1, r2):
    grad_x = torch.autograd.grad(psi_val, r1, create_graph=True)[0]
    grad_y = torch.autograd.grad(psi_val, r2, create_graph=True)[0]

    # Laplacians
    laplacian_x = sum(torch.autograd.grad(grad_x[i], r1, retain_graph=True, create_graph=True)[0][i] for i in range(3))
    laplacian_y = sum(torch.autograd.grad(grad_y[i], r2, retain_graph=True, create_graph=True)[0][i] for i in range(3))

    ke = - 0.5 * (laplacian_x + laplacian_y)

    return - 2 / torch.norm(r1) - 2 / torch.norm(r2) + 1 / torch.norm(r1 - r2) + ke
"""

def local_energy(psi_val, r1, r2):
    grad_r1 = torch.autograd.grad(psi_val, r1, create_graph=True)[0]
    grad_r2 = torch.autograd.grad(psi_val, r2, create_graph=True)[0]

    lap_r1 = torch.autograd.grad(grad_r1, r1, grad_outputs=torch.ones_like(grad_r1), create_graph=True)[0]
    lap_r2 = torch.autograd.grad(grad_r2, r2, grad_outputs=torch.ones_like(grad_r2), create_graph=True)[0]

    laplacian = lap_r1.sum() + lap_r2.sum()
    ke = -0.5 * laplacian

    potential = -2 / torch.norm(r1) - 2 / torch.norm(r2) + 1 / torch.norm(r1 - r2)

    return ke + potential


In [6]:
L = 1
r1 = torch.rand(3, requires_grad=True) * 2 * L - L
r2 = torch.rand(3, requires_grad=True) * 2 * L - L #random number from -L to L
E = 0
E2 = 0
Eln_average = 0
ln_average = 0
rejection_ratio = 0
step = 0
max_steps = 500
N = 10000
dlap_dalpha = 0
xs = []
ys = []
psi_vals = []

for i in tqdm(range(N)):
    flag = False
    chose = np.random.rand()
    step = step + 1
    if chose < 0.5:
        r1_trial = r1 + 0.5 * (torch.rand(3) * 2 * L-L)
        r2_trial = r2
    else:
        r2_trial = r2 + 0.5 * (torch.rand(3) * 2 * L-L)
        r1_trial = r1


    psi_val = psi(r1, r2, *alphas)
    psi_val_trial = psi(r1_trial, r2_trial, *alphas)
    

    if psi_val_trial.item() >= psi_val.item():
        r1 = r1_trial
        r2 = r2_trial
        flag = True
    
    else:
        dummy = np.random.rand()
        if dummy < psi_val_trial.item() / psi_val.item():
            r1 = r1_trial
            r2 = r2_trial
        else:
            rejection_ratio += 1./N
            
    if step > max_steps:
        xs.append(r1)
        ys.append(r2)

        if flag:
            psi_vals.append(psi_val_trial)
        else:
            psi_vals.append(psi_val)
        """
        local_E = local_energy(psi_val_trial, r1, r2)
        E += local_E / (N - max_steps)
        E2 += local_E ** 2 / (N - max_steps)
        dlap_dalpha += torch.tensor(torch.autograd.grad(local_E, alphas, retain_graph=True)) / (N - max_steps)
        """





100%|██████████| 10000/10000 [00:05<00:00, 1786.93it/s]


In [7]:
def batched_psi(x, y, alpha_1, alpha_2, alpha_3, alpha_4):
    r1 = torch.norm(x, dim=1)
    r2 = torch.norm(y, dim=1)
    r12 = torch.norm(x - y, dim=1)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

In [8]:
alpha_1 = torch.tensor(1.013, dtype=torch.float64, requires_grad=True)
alpha_2 = torch.tensor(0.2119, dtype=torch.float64, requires_grad=True)
alpha_3 = torch.tensor(0.1406, dtype=torch.float64, requires_grad=True)
alpha_4 = torch.tensor(0.003, dtype=torch.float64, requires_grad=True)

def batched_psi(x, y):
    r1 = torch.norm(x, dim=1)
    r2 = torch.norm(y, dim=1)
    r12 = torch.norm(x - y, dim=1)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3


In [ ]:
vmap(psi)

In [38]:
t = batched_psi(torch.stack(xs), torch.stack(ys))

In [ ]:
torch.autograd.grad(t, torch.stack(xs[:5]), is_grads_batched=True)

In [16]:
from torch.func import vmap, grad

In [33]:
batched_grad_f = vmap(grad(batched_psi))

In [59]:
jacobian = torch.autograd.functional.jacobian(batched_psi, (torch.stack(xs), torch.stack(ys)))

In [60]:
jacobian[0].shape

torch.Size([9500, 9500, 3])

In [63]:
from torch.func import grad, vmap

In [162]:
import torch
from torch.autograd.functional import jacobian

# Original scalar-valued psi
def psi(X):
    x = X[:3]
    y = X[3:6]
    alpha_1, alpha_2, alpha_3, alpha_4 = X[6:]
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

In [181]:

def third_partial_derivatives(scalar_func, inp):
    return jacobian(
        lambda x: jacobian(
            lambda y: jacobian(scalar_func, y), 
            x
        ), 
        inp
    )

In [171]:
t = vmap(psi)

In [9]:
inputs = []
for i in range(len(xs)):
    inputs.append(torch.tensor([*xs[i], *ys[i], alpha_1, alpha_2, alpha_3, alpha_4]))

In [10]:
inputs = torch.stack(inputs)

In [11]:
inputs[0]

tensor([-0.3750, -0.2830,  0.4478,  0.7029,  0.1027, -0.2183,  1.0130,  0.2119,
         0.1406,  0.0030], dtype=torch.float64)

In [176]:
t(inputs)

tensor([0.0039, 0.0046, 0.0035,  ..., 0.0008, 0.0013, 0.0020],
       dtype=torch.float64)

In [100]:
s = vmap(grad(psi))

In [120]:
def fs(x):
    return torch.exp(x[0] + x[1])

In [121]:
l = vmap(grad(grad(fs)))

In [127]:
l = vmap(grad(grad(fs)))

In [114]:
grad(grad(fs))(torch.tensor(3.))

tensor(20.0855)

In [ ]:
t = vmap(hessian(psi))
E = torch.diagonal(hess_full[0]).sum()
grad(E, alphas)

In [145]:
hess_full = t(inputs)

In [ ]:
hess_full[0].shape 

torch.Size([6, 6])

In [155]:
E = torch.diagonal(hess_full[0]).sum()

In [ ]:
lap_alpha = jacobian(hessian(psi))

In [ ]:
lap_alpha(inputs[0])

## Verifying the analytical expressions for the deriviatives

In [78]:
from torch.autograd.functional import jacobian, hessian

In [123]:
def psi_a(X):
    x = X[:3]
    y = X[3:6]
    alpha_1, alpha_2, alpha_3, alpha_4 = X[6:]
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1

In [128]:
def psi_a_first(X):
    x = X[:3]
    y = X[3:6]
    alpha_1, alpha_2, alpha_3, alpha_4 = X[6:]
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)
    r1_ = x[0] + x[1] + x[2]
    r2_ = y[0] + y[1] + y[2]

    term1 = torch.exp(-2 * (r1 + r2))

    return -2 * term1 * (r1_/r1 + r2_/r2)

In [150]:
def psi_a_(X):
    x = X[:3]
    y = X[3:6]
    alpha_1, alpha_2, alpha_3, alpha_4 = X[6:]
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)
    r1_ = x[0] + x[1] + x[2]
    r2_ = y[0] + y[1] + y[2]

    term1 = torch.exp(-2 * (r1 + r2))

    return 2 * term1 * (2 * (r1_.pow(2) / r1.pow(2)) + 2 * (r2_.pow(2) / r2.pow(2)) - 3/r1 - 3/r2 + r1_.pow(2)/r1.pow(3) + r2_.pow(2) / r2.pow(3))

In [156]:
def psi_b(X):
    x = X[:3]
    y = X[3:6]
    alpha_1, alpha_2, alpha_3, alpha_4 = X[6:]
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term2

In [188]:
def psi_b_first(X):
    x = X[:3]
    y = X[3:6]
    alpha_1, alpha_2, alpha_3, alpha_4 = X[6:]
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)
    r12_ = sum(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return (torch.exp(-alpha_1 * r12)/2) * (r12_/r12) * (1 - alpha_1 * r12)

def psi_b_first_(X):
    x = X[:3]
    y = X[3:6]
    alpha_1, alpha_2, alpha_3, alpha_4 = X[6:]
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)
    r12_ = sum(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return (torch.exp(-alpha_1 * r12)/2) * (r12_/r12) * (alpha_1 * r12 - 1)

In [ ]:
jac = jacobian(psi_a, inputs[33])

In [161]:
jac

tensor([ 0.0395,  0.0076, -0.0538, -0.0395, -0.0076,  0.0538, -0.2628,  0.0000,
         0.0000,  0.0000], dtype=torch.float64)

In [162]:
sum(jac[:3])

tensor(-0.0067, dtype=torch.float64)

In [200]:
sample = 33

jac = jacobian(psi_b, inputs[sample])
print(f"Jacobian is {sum(jac[:3])}")
print(f"Analytical is {psi_b_first(inputs[sample])}")

Jacobian is -0.00392798153454757
Analytical is -0.003927981534547572


In [201]:
hes = hessian(psi_b, inputs[sample])
jac = jacobian(psi_b_first_, inputs[sample])

print(sum(torch.diag(hes)[:6]))
#print(sum(jac[3:6]))

tensor(-0.0786, dtype=torch.float64)


In [202]:
sum(jacobian(psi_b_first_, inputs[sample])[3:6]) + sum(jacobian(psi_b_first, inputs[sample])[:3])

tensor(-0.1673, dtype=torch.float64)

In [226]:
def psi(X):
    x = X[:3]
    y = X[3:6]
    alpha_1, alpha_2, alpha_3, alpha_4 = X[6:]
    alpha_2 = torch.tensor(3.14)
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

In [227]:
def grad_hes(inputs):
    return hessian(psi, inputs)

In [229]:
grad_hes(inputs[232])

tensor([[-5.1090e-03, -1.8056e-03,  5.0205e-04,  7.8134e-04,  3.7859e-04,
         -9.4882e-04,  4.2517e-05,  0.0000e+00,  6.7591e-07, -7.2551e-05],
        [-1.8056e-03,  2.0511e-02, -5.9669e-03, -1.9461e-02, -9.4284e-04,
          1.1649e-02,  2.9095e-03,  0.0000e+00, -3.6415e-04,  1.1880e-03],
        [ 5.0205e-04, -5.9669e-03, -3.7575e-03,  4.5048e-03, -1.4176e-04,
         -3.2784e-03, -8.2504e-04,  0.0000e+00,  9.9698e-05, -2.8344e-04],
        [ 7.8134e-04, -1.9461e-02,  4.5048e-03,  1.5181e-02,  2.5460e-03,
         -1.6832e-02, -2.3922e-03,  0.0000e+00,  1.3215e-03, -1.0912e-03],
        [ 3.7859e-04, -9.4284e-04, -1.4176e-04,  2.5460e-03, -1.2407e-02,
         -1.3907e-03, -6.9872e-04,  0.0000e+00,  2.7419e-04, -7.9724e-05],
        [-9.4882e-04,  1.1649e-02, -3.2784e-03, -1.6832e-02, -1.3907e-03,
         -1.5762e-03,  1.6138e-03,  0.0000e+00, -8.6079e-04,  6.7046e-04],
        [ 4.2517e-05,  2.9095e-03, -8.2504e-04, -2.3922e-03, -6.9872e-04,
          1.6138e-03,  5.5859e-0

In [228]:
jacobian(grad_hes, inputs[421]).any()

tensor(False)

In [181]:
def grad_psi_b(inputs):
    return jacobian(psi_b, inputs)

In [182]:
jac = jacobian(grad_psi_b, inputs[sample])

In [183]:
jac

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=torch.float64)

In [151]:
sample = 141

hes = hessian(psi_a, inputs[sample])
print(f"Hessian is {sum(torch.diag(hes)[:6])}")
print(f"Analytical is {psi_a_(inputs[sample])}")

Hessian is -0.3674722041861118
Analytical is 0.12678009266415927


In [112]:
inputs[sample]

tensor([-0.3664,  0.7710,  0.6767,  0.7932,  0.4680,  0.0132,  1.0130,  0.2119,
         0.1406,  0.0030], dtype=torch.float64)

In [98]:
psi_a_(inputs[21])

tensor(-0.0060, dtype=torch.float64)